In [12]:
import generate
import numpy as np
import plotly.graph_objects as go
import polars as pl
from edm.ccm import find_best_embedding, scan_best_embedding
from edm.embedding import lagged_embed
from edm.simplex_projection import pairwise_distance, simplex_projection, topk
from tinygrad import Tensor
from tinygrad.dtype import dtypes

In [2]:
sigma, rho, beta = 10, 28, 8 / 3
dt = 0.01
t_max = 100

X0 = np.array([0.1, 0.0, 0.0])

In [3]:
r, x = generate.lorenz(sigma, rho, beta, X0, dt, t_max)

x = Tensor(x, dtype=dtypes.float32)
index = 300
k = 100

D = pairwise_distance(x)
indices, _ = topk(D[index].numpy(), k + 1, largest=False)
indices = indices[1:]

layout = go.Layout(
    scene=dict(xaxis=dict(title="X"), yaxis=dict(title="Y"), zaxis=dict(title="Z")),
    width=800,
    height=800,
)

x = x.numpy()

data = [
    go.Scatter3d(
        name="lorenz",
        x=x[:, 0],
        y=x[:, 1],
        z=x[:, 2],
        mode="markers",
        marker=dict(size=2),
    ),
    go.Scatter3d(
        name="knn",
        x=x[indices][:, 0],
        y=x[indices][:, 1],
        z=x[indices][:, 2],
        mode="markers",
        marker=dict(size=5, color="red"),
    ),
    go.Scatter3d(
        name="target",
        x=[x[index][0]],
        y=[x[index][1]],
        z=[x[index][2]],
        mode="markers",
        marker=dict(size=5, color="green"),
    ),
]

fig = go.Figure(data=data, layout=layout)
fig.show()

In [4]:
r, x = generate.lorenz(sigma, rho, beta, X0, dt, t_max)

x = Tensor(x, dtype=dtypes.float32)
index = 300
k = 100
exclusion_radius = 5

D = pairwise_distance(x).numpy()
N = D.shape[0]
mask = np.ones(N, dtype=bool)
mask[index] = False
mask[max(0, index - exclusion_radius) : min(N, index + exclusion_radius + 1)] = False

indices_masked, _ = topk(D[index][mask], k, largest=False)
indices = np.arange(N)[mask][indices_masked]

layout = go.Layout(
    scene=dict(xaxis=dict(title="X"), yaxis=dict(title="Y"), zaxis=dict(title="Z")),
    width=800,
    height=800,
)

x = x.numpy()

data = [
    go.Scatter3d(
        name="lorenz",
        x=x[:, 0],
        y=x[:, 1],
        z=x[:, 2],
        mode="markers",
        marker=dict(size=2),
    ),
    go.Scatter3d(
        name="knn",
        x=x[indices][:, 0],
        y=x[indices][:, 1],
        z=x[indices][:, 2],
        mode="markers",
        marker=dict(size=5, color="red"),
    ),
    go.Scatter3d(
        name="target",
        x=[x[index][0]],
        y=[x[index][1]],
        z=[x[index][2]],
        mode="markers",
        marker=dict(size=5, color="green"),
    ),
    go.Scatter3d(
        name="masked",
        x=x[~mask][:, 0],
        y=x[~mask][:, 1],
        z=x[~mask][:, 2],
        mode="markers",
        marker=dict(size=2, color="black"),
    ),
]

fig = go.Figure(data=data, layout=layout)
fig.show()

In [5]:
t, X = generate.lorenz(sigma, rho, beta, X0, dt, t_max)
embedding = lagged_embed(X[:, 1], 2, 3)
y = X[: len(embedding), 2]
t = np.arange(100)
observations, predictions = simplex_projection(embedding, y, t, 1)

layout = go.Layout(
    xaxis=dict(title="t"),
    yaxis=dict(title="x"),
    width=800,
    height=800,
)

data = [
    go.Scatter(name="observation", x=t, y=observations),
    go.Scatter(
        name="prediction",
        x=t,
        y=predictions,
        line=dict(color="red"),
    ),
]

fig = go.Figure(data=data, layout=layout)
fig.show()

In [6]:
def compute_rho(y, y_hat):
    return np.corrcoef(y, y_hat)[0, 1]


compute_rho(observations, predictions)

np.float64(0.8577230727399164)

In [15]:
df = pl.read_csv("../data/preprocessed_2090_combined_timeseries.csv")

x = df["YML027W"].to_numpy()

tau_list = list(range(1, 5))
e_list = list(range(1, 6))
Tp = 7

best_tau, best_e, best_rho = find_best_embedding(x, tau_list, e_list, Tp)
print(f"Best embedding: tau={best_tau}, e={best_e}, rho={best_rho}")

Best embedding: tau=1, e=2, rho=1.0


/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2914: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning:

divide by zero encountered in divide

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning:

invalid value encountered in multiply

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning:

invalid value encountered in divide



In [9]:
result = scan_best_embedding(x, tau_list, e_list, Tp)
result

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2914: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning:

divide by zero encountered in divide

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning:

invalid value encountered in multiply

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning:

invalid value encountered in divide

/Users/temma/ghq/LabRotations/CHAOS/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning:

invalid value encountered in divide



tau,e,rho
i64,i64,f64
1,1,NaN
1,2,1.0
1,3,0.970869
1,4,0.978089
1,5,0.960647
…,…,…
4,1,NaN
4,2,1.0
4,3,1.0
